In [1]:
import pandas
import math
import re
import matplotlib.pyplot as plt
import numpy
import scipy
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
SKEMPI = pandas.read_csv("skempi_v2.csv", sep = ";")
PDBList = ("1JTG", "5E9D", "3SZK", "3BN9", "2J0T", "1KTZ", "3SGB", "3MZG", "1DAN", "1VFB", "1OGA")
PDBList2 = ("1JTG_A_B", "5E9D_AB_CDE", "3SZK_AB_C", "3BN9_B_CD", "2J0T_A_D",
           "1KTZ_A_B", "3SGB_E_I", "3MZG_A_B", "1DAN_HL_UT", "1VFB_AB_C", "1OGA_ABC_DE")
PDBList3 = ("1PPF_E_I", "1AO7_ABC_DE", "1CHO_EFG_I", "1R0R_E_I", "1MHP_HL_A", "3HFM_HL_Y",
           "1KNE_A_P", "2JEL_LH_P", "2FTL_E_I", "1IAR_A_B")

filtered_SKEMPI = SKEMPI[SKEMPI["#Pdb"].isin(PDBList3)]
filtered_df = filtered_SKEMPI[filtered_SKEMPI['Mutation(s)_cleaned'].str.len() <= 7]
filtered_df.index = range(0, len(filtered_df))

In [53]:
def extract_key(s):
    match = re.match(r"(.)(.)(\d+)(.)", s)
    if match:
        amino, chain, pos, mut = match.groups()
        return (amino, int(pos), mut)  # Sort first by character Y, then by number
    return ("", 0)  # Fallback, if needed
custom_order = {'H': 0, 'L': 1, 'U': 2, 'T': 3}  # Lower number means higher priority

def extract_key_sp(s):
    match = re.match(r"(.)(.)(\d+)(.)", s)  # First char, sorting char, number, last char
    if match:
        _, y, num, _ = match.groups()
        return (custom_order.get(y, 99), int(num))  # Default rank 99 if char not found
    return (99, 0)  # Fallback case

In [32]:
toA = 0
for each in filtered_df['Mutation(s)_cleaned']:
    if (each[-1] == "A"):
        toA += 1
print(toA)
len(filtered_df)

202


1071

In [43]:
numineach = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

i = 0
while (i < len(filtered_df)):
    pdb = filtered_df['#Pdb'][i]
    numineach[PDBList3.index(pdb)] += 1
    i += 1
print(numineach)

[203, 146, 205, 205, 87, 96, 13, 44, 32, 40]


In [60]:
regions = [0, 0, 0, 0, 0]
regionlist = ["COR", "INT", "RIM", "SUP", "SUR"]

i = 0
while (i < len(filtered_df)):
    reg = filtered_df['iMutation_Location(s)'][i]
    regions[regionlist.index(reg)] += 1
    i += 1
print(regions)

[579, 24, 297, 113, 58]


In [64]:
mut_type_dict = {
    'A': 0, 'V': 0, 'I': 0, 'L': 0, 'M': 0, 
    'F': 0, 'Y': 0, 'W': 0, 'S': 1, 'T': 1, 
    'N': 1, 'Q': 1, 'R': 2, 'H': 2, 'K': 2, 
    'D': 3, 'E': 3, 'C': 4, 'G': 4, 'P': 4
}
color_dict = {
    0 : "Same Type", 1 : "Hydrophobic to Polar", 2 : "Hydrophobic to Positive", 3 : "Hydrophobic to Negative", 4 : "Hydrophobic to Other",
    10 : "Any to Hydrophobic", 11 : "Same Type", 12 : "Polar to Positive", 13 : "Polar to Negative", 14 : "Any to Other",
    20 : "Any to Hydrophobic", 21 : "Positive to Polar", 22 : "Same Type", 23 : "Positive to Negative", 24 : "Any to Other",
    30 : "Any to Hydrophobic", 31 : "Negative to Polar", 32 : "Negative to Positive", 33 : "Same Type", 34 : "Any to Other",
    40 : "Any to Hydrophobic", 41 : "Hydrophobic to Polar", 42 : "Hydrophobic to Positive", 43: "Hydrophobic to Negative", 44: "Same Type"
}


i = 0
while (i < len(filtered_df)):
    this = filtered_df['Mutation(s)_cleaned'][i]
    match = extract_key(this)
    print(match)
    i += 1
    fro = mut_type_dict[match[0]]
    to = mut_type_dict[match[2]]
    mut_type = (fro * 10) + to
    print(mut_type)
    mut_types.append(color_dict[mut_type])

('I', 5, 'A')
0
('I', 5, 'R')
2
('T', 6, 'A')
10
('T', 6, 'D')
13
('Q', 8, 'A')
10
('Q', 8, 'R')
12
('E', 9, 'Q')
31
('E', 9, 'K')
32
('E', 9, 'A')
30
('I', 11, 'A')
0
('K', 12, 'S')
21
('K', 12, 'E')
23
('T', 13, 'A')
10
('T', 13, 'D')
13
('N', 15, 'A')
10
('N', 15, 'D')
13
('S', 16, 'A')
10
('S', 16, 'D')
13
('E', 19, 'A')
30
('E', 19, 'R')
32
('R', 53, 'Q')
21
('K', 77, 'A')
20
('K', 77, 'E')
23
('Q', 78, 'A')
10
('Q', 78, 'E')
13
('R', 81, 'A')
20
('R', 81, 'E')
23
('F', 82, 'A')
0
('F', 82, 'D')
3
('K', 84, 'A')
20
('K', 84, 'D')
23
('R', 85, 'A')
20
('R', 85, 'E')
23
('R', 88, 'Q')
21
('R', 88, 'D')
23
('R', 88, 'A')
20
('N', 89, 'A')
10
('N', 89, 'R')
12
('W', 91, 'A')
0
('W', 91, 'D')
3
('F', 2, 'Y')
0
('F', 2, 'W')
0
('Q', 3, 'K')
12
('Q', 4, 'K')
12
('E', 5, 'D')
33
('E', 5, 'Q')
31
('V', 6, 'F')
0
('T', 7, 'N')
11
('T', 7, 'S')
11
('P', 11, 'E')
43
('N', 12, 'D')
13
('R', 17, 'G')
24
('R', 17, 'K')
22
('K', 24, 'E')
23
('K', 27, 'E')
23
('T', 34, 'N')
11
('T', 34, 'Q')
11
('